In [1]:
push!(LOAD_PATH, "..")
using ASE, Potentials, MatSciPy, AtomsInterface, TestAtoms, TightBinding
using PyPlot
import AtOptim
import AtPrecon

In [2]:
# reference configuration
at = bulk("Al"; cubic=true)
at = repeat(at, (5, 5, 1))
set_pbc!(at, (false, false, false))
X0 = positions(at);
# jiggle the positions
set_positions!(at, X0+0.01 * rand(size(X0)))
# create a calculator
r0 = rnn("Al")
rcut = 2.5 * r0
# calc = MatSciPy.PairCalculator( SWCutoff( LennardJonesPotential(r0, 1.0), rcut, 1.0 ) )
calc = TightBinding.ToyTB.ToyTBModel(r0=r0, rcut=rcut)
TestAtoms.test_potentialenergy(calc, at)
;

-----------------------------
  p | error 
----|------------------------
  2 | 2.5104853e-03 
  3 | 2.5085740e-04 
  4 | 2.5083939e-05 
  5 | 2.5067275e-06 
  6 | 2.6688192e-07 
  7 | 3.4464724e-07 
  8 | 3.1251414e-06 
  9 | 3.5275321e-05 
 10 | 3.8457008e-04 
-----------------------------


In [ ]:
at = repeat(bulk("Al"; cubic=true), (10,10,2))
println(" #at = ", length(at))
# precon = AtPrecon.IdPrecon()
precon = AtPrecon.ExpPrecon(at, calc)
X0 = positions(at)
srand(12345)
set_positions!(at, X0+0.05 * rand(size(X0)))
@time AtOptim.minimise!(at, calc, 1e-4; precon=precon, disp=3, maxnit=100);

 #at = 4800

In [7]:
at = repeat(bulk("Al"; cubic=true), (3, 3, 3))
println(" #at = ", length(at))
X0 = positions(at)
srand(12345)
set_positions!(at, X0+0.05 * rand(size(X0)))
calc = TightBinding.ToyTB.ToyTBModel(r0=r0, rcut=rcut)
calc.nkpoints = (0,0,1)
@time precon = AtPrecon.ExpPrecon(at, calc)
# precon = AtPrecon.IdPrecon()
AtOptim.minimise!(at, calc, 1e-4; precon=precon, disp=3, maxnit=100);

 #at = 108
  3.487476 seconds (64.35 M allocations: 1.790 GB, 8.32% gc time)
-------|---------------------------------------------------------
 nit   |   ΔE        |∇E|∞     |Δx|∞      α        #E 
-------|---------------------------------------------------------
     1 | -7.41e+01  2.44e-02  0.00e+00  1.00e+00      1 
     2 | -1.12e-02  6.92e-03  8.38e-02  1.85e-01      3 
     3 | -1.12e-03  3.53e-03  1.24e-01  1.70e-01      4 
     4 | -3.93e-04  2.06e-03  1.03e-01  1.80e-01      5 
     5 | -3.18e-04  1.59e-03  1.33e-01  2.84e-01      6 
     6 | -2.82e-04  1.43e-03  2.60e-01  3.63e-01      7 
     7 | -9.68e-05  1.42e-03  1.26e-01  1.05e-01      8 
     8 | -1.03e-05  1.43e-03  4.47e-03  1.05e-02      9 
     9 | -1.04e-06  1.43e-03  7.35e-04  1.05e-03     10 
    10 | -1.04e-07  1.43e-03  5.87e-05  1.05e-04     11 
    11 | -1.04e-08  1.43e-03  7.06e-06  1.05e-05     12 
    12 | -1.04e-09  1.43e-03  8.44e-07  1.05e-06     13 
    13 | -1.04e-10  1.43e-03  7.64e-08  1.05e-07    

LoadError: LoadError: ls_armijo! : alpha < alpha_min 
while loading In[7], in expression starting on line 10

  14 | -1.04e-11  1.43e-03  3.81e-09  1.05e-08     15 


In [13]:
at = repeat(bulk("Al"; cubic=true), (3, 3, 3))
println(" #at = ", length(at))
X0 = positions(at)
srand(12345)
set_positions!(at, X0+0.05 * rand(size(X0)))
calc = TightBinding.ToyTB.ToyTBModel(r0=r0, rcut=rcut)
calc.nkpoints = (0,0,1)
precon = AtPrecon.ExpPrecon(at, calc); precon.cstab = 0.1
# precon = AtPrecon.IdPrecon()
alpha = 0.1
X = copy(X0)
for n = 1:20
    frc = forces(at, calc)
    @printf(" |frc|∞ = %4.2e \n", vecnorm(frc, Inf))
    X += alpha * (frc / precon)
    set_positions!(at, X)
end


 #at = 108
 |frc|∞ = 1.82e-02 
 |frc|∞ = 1.09e-02 
 |frc|∞ = 4.30e-03 
 |frc|∞ = 2.13e-03 
 |frc|∞ = 1.10e-03 
 |frc|∞ = 6.17e-04 
 |frc|∞ = 4.41e-04 
 |frc|∞ = 3.16e-04 
 |frc|∞ = 2.44e-04 
 |frc|∞ = 1.90e-04 
 |frc|∞ = 1.65e-04 
 |frc|∞ = 1.48e-04 
 |frc|∞ = 1.42e-04 
 |frc|∞ = 1.42e-04 
 |frc|∞ = 1.43e-04 
 |frc|∞ = 1.45e-04 
 |frc|∞ = 1.49e-04 
 |frc|∞ = 1.54e-04 
 |frc|∞ = 1.60e-04 
 |frc|∞ = 1.68e-04 
